## Example of querying a set of documents with sources using the Hugging Face Text Generation Inference server with Llama2, Langchain and a custom prompt

In [1]:
!pip install langchain pypdf sentence-transformers chromadb==0.3.29 text_generation


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Set the Inference server url (replace with your own address)

In [2]:
inference_server_url_flant5_xl = "http://hf-flant5-xl.llm-model-collection.svc.cluster.local:3000/"

inference_server_url_mistral = "http://hf-mistral-7b.llm-model-collection.svc.cluster.local:3000/"

COLLECTION_NAME = "documents_test"


### Load some data from the folder where we have stored the PDF documentation

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader

pdf_folder_path = 'rhods-doc'

loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

### Split the data in chunks large enough to have meaningful answers, and some overlap not to miss anything

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024, chunk_overlap = 40)
all_splits = text_splitter.split_documents(docs)

### Store the data as embeddings in a vector database (Chroma)

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=HuggingFaceEmbeddings())

/opt/app-root/lib64/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Test data retrieving

In [6]:
question = "How do I create a Data Science Project"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='-\n, and must start and end with an alphanumeric\ncharacter.\n5\n. \nEnter a \ndescription\n for your data science project.\n6\n. \nClick \nCreate\n.\nA project details page opens. From here, you can create workbenches, add cluster storage, and\nadd data connections to your project.\nVerification\nThe data science project that you created is displayed on the \nData science projects\n page.\nCHAPTER 4. CREATING A DATA SCIENCE PROJECT\n9', metadata={'source': 'rhods-doc/red_hat_openshift_data_science_self-managed-1.32-getting_started_with_red_hat_openshift_data_science_self-managed-en-us.pdf', 'page': 12}),
 Document(page_content='-\n, and must start and end with an alphanumeric\ncharacter.\n5\n. \nEnter a \ndescription\n for your data science project.\n6\n. \nClick \nCreate\n.\nA project details page opens. From here, you can create workbenches, add cluster storage, and\nCHAPTER 3. WORKING ON DATA SCIENCE PROJECTS\n11', metadata={'source': 'rhods-doc/red_hat_open

### Create the chain

In [7]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# NOTE: This template syntax is specific to Llama2
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
You will be given a question you need to answer, and a context to provide you with information. You must answer the question based as much as possible on this context.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Question: {question}
Context: {context} [/INST]
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm_flant5_xl = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_flant5_xl,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.175,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain_llm_flant5 = RetrievalQA.from_chain_type(llm_flant5_xl,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True)



llm_mistral = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_mistral,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.175,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain_mistral = RetrievalQA.from_chain_type(llm_mistral,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True)



/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [8]:
question = "How do I create a Data Science Project?"
result = qa_chain_llm_flant5({"query": question})

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 To start your data science work, create a data science project. Creating a project helps you organize your work in one place. You can also enhance the capabilities of your data science project by adding workbenches, adding storage to your project’s cluster, adding data connections, and adding model servers.

### Launch the query

In [9]:
question = "How do I create a Data Science Project?"
result = qa_chain_mistral({"query": question})

Creating a data science project involves several steps. Firstly, you need to log into Red Hat OpenShift Data Science. Then, from the dashboard, navigate to the "Data Science Projects" page. Here, you can click on the "Create data science project" button to open the "Create a data science project" dialog box. In this dialog box, enter a name for your project. Optionally, you can edit the resource name, which must consist only of lowercase alphanumeric characters and hyphens. Once you have entered the required information, click on the "Create" button to create your data science project. After creating the project, you can further customize it by adding workbenches, cluster storage, data connections, and model servers.

### Print the sources

In [10]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

rhods-doc/red_hat_openshift_data_science_self-managed-1.32-getting_started_with_red_hat_openshift_data_science_self-managed-en-us.pdf
rhods-doc/red_hat_openshift_data_science_self-managed-1.32-working_on_data_science_projects-en-us.pdf
